# `--- Day 22: Sporifica Virus ---`

In [1]:
input_lines = open('input.txt').readlines()
test_lines = '''..#
#..
...'''.splitlines()

In [2]:
DIRECTIONS = 'NESW' # in clockwise order
MOVES = {'N': (-1, 0),
         'E': (0, 1),
         'S': (1, 0),
         'W': (0, -1)}

In [3]:
class Sporifica(object):
    def __init__(self, lines):
        self.grid = {}
        height = len(lines)
        for row, l in enumerate(lines):
            l = l.strip()
            width = len(l)
            for col, c in enumerate(l):
                self.grid[(row, col)] = c

        self.current_row = width // 2 # grid is odd number of chars wide
        self.current_col = height // 2
        self.directions = DIRECTIONS # directions[0] is current direction
        self.infections = 0
        
    def direction(self):
        return self.directions[0]
        
    def turn_right(self):
        self.directions = self.directions[1:] + self.directions[0]
        return self
    
    def turn_left(self):
        self.directions = self.directions[-1] + self.directions[0:-1]
        return self
    
    def get_current(self):
        return self.get(self.current_row, self.current_col)
    
    def set_current(self, s):
        self.grid[(self.current_row, self.current_col)] = s
        
    def get(self, r, c):
        if (r, c) not in self.grid:
            self.grid[(r, c)] = '.'
        return self.grid[(r, c)]
    
    def burst(self):
        if self.get_current() == '#':
            self.turn_right()
            self.set_current('.') # clean the node
        else:
            self.turn_left()
            self.set_current('#') # infect the node
            self.infections += 1
        step_r, step_c = MOVES[self.direction()]
        self.current_row += step_r
        self.current_col += step_c
        return self

    def burst2(self):
        cur = self.get_current()
        if cur == '#': # infected -> flagged
            self.turn_right()
            self.set_current('F')
        elif cur == 'F': # flagged -> clean
            self.turn_right()
            self.turn_right()
            self.set_current('.')
        elif cur == '.': # clean -> weakened
            self.turn_left()
            self.set_current('W')
        elif cur == 'W': # weakened -> infected
            self.set_current('#')
            self.infections += 1
        else:
            raise ValueError(f'Unexpected state {cur}')
        step_r, step_c = MOVES[self.direction()]
        self.current_row += step_r
        self.current_col += step_c
        return self
        
    def __str__(self):
        s = ''
        keys = self.grid.keys()
        rows, cols = zip(*keys)
        r = min(rows)
        while r <= max(rows):
            c = min(cols)
            while c <= max(cols):
                if r == self.current_row and c == self.current_col:
                    L, R = list('[]')
                else:
                    L = R = ' '
                s += L + self.get(r, c) + R
                c += 1
            s += '\n'
            r += 1
        return s
           
    __repr__ = __str__

In [4]:
%%timeit -n 1 -r 1
s = Sporifica(input_lines)

for i in range(10000):
    s.burst()
    
print(f'part 1 answer: {s.infections}')

part 1 answer: 5246
23.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%%timeit -n 1 -r 1
s2 = Sporifica(input_lines)

for i in range(10000000):
    s2.burst2()
    
print(f'part 2 answer {s2.infections}')

part 2 answer 2512059
20.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# timings

## Dell XPS13 i7
```
part 1 answer: 5246
20.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
part 2 answer 2512059
31.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
```

## MacBook Pro
```
part 1 answer: 5246
23.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
part 2 answer 2512059
20.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
```